In [1]:
# use CEASELESS Kernel on adrien local laptop

import os
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import cartopy.crs as ccrs # => to add in Seastar env
import seastar as ss
from seastar.utils.tools import dotdict

from scipy import interpolate, stats # => to add in Seastar env ?
from scipy.optimize import least_squares

# from seastar.gmfs.doppler import *

In [2]:
level1 = xr.Dataset(
    data_vars=dict(
            CentralWavenumber=( [],270 ),
            CentralFreq=( [], 13.5 * 10**9 ),
            IncidenceAngleImage=( ['across','along','Antenna'], np.full([9,11,4], 30) ),
            AntennaAzimuthImage=(['across', 'along', 'Antenna'],
                           np.stack((np.full([9, 11], 45),
                                     np.full([9, 11], 90),
                                     np.full([9, 11], 90),
                                     np.full([9, 11], 135)
                                     ), axis=-1
                                    )
                           ),
            LookDirection=(['across', 'along', 'Antenna'],
                           np.stack((np.full([9, 11], 45),
                                     np.full([9, 11], 90),
                                     np.full([9, 11], 90),
                                     np.full([9, 11], 135)
                                     ), axis=-1
                                    )
                           ),
            Polarization=(['across', 'along','Antenna'],
                          np.stack((np.full([9, 11], 'VV'),
                                    np.full([9, 11], 'VV'),
                                    np.full([9, 11], 'HH'),
                                    np.full([9, 11], 'VV')
                                    ), axis=-1
                                   )
                          ),
#             Sigma0=( ['across','along','Antenna'], np.full([9,11,4], 1.01) ),
#             dsig0=( ['across','along','Antenna'], np.full([9,11,4], 0.05) ),
#             RVL=( ['across','along','Antenna'], np.full([9,11,4], 0.5) ),
#             drvl=( ['across','along','Antenna'], np.full([9,11,4], 0.01) ),
        ),
    coords=dict(
            across=np.arange(0,9),
            along=np.arange(0,11),
            Antenna=['Fore','MidV','MidH','Aft'],
        ),
)
level1 = level1.set_coords([
    'CentralWavenumber',
    'CentralFreq',
    'IncidenceAngleImage',
    'AntennaAzimuthImage',
    'LookDirection',
    'Polarization', 
])


In [3]:
geo = xr.Dataset(
        data_vars=dict(
            WindSpeed=(['across','along'], np.full([9,11], 10) ),
            WindDirection=(['across','along'], np.full([9,11], 150) ),
            CurrentVelocity=(['across','along'], np.full([9,11], 1) ),
            CurrentDirection=(['across','along'], np.full([9,11], 150) ),
        ),
        coords=dict(
            across=np.arange(0,9),
            along=np.arange(0,11),
        ),
)
geo

<xarray.Dataset>
Dimensions:           (across: 9, along: 11)
Coordinates:
  * across            (across) int64 0 1 2 3 4 5 6 7 8
  * along             (along) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    WindSpeed         (across, along) int64 10 10 10 10 10 10 ... 10 10 10 10 10
    WindDirection     (across, along) int64 150 150 150 150 ... 150 150 150 150
    CurrentVelocity   (across, along) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    CurrentDirection  (across, along) int64 150 150 150 150 ... 150 150 150 150

In [4]:
gmf=dotdict({'nrcs': dotdict({'name': 'nscat4ds'})})
gmf['doppler'] = dotdict({'name': 'mouche12'})

In [5]:
level1['Sigma0'] = ss.gmfs.nrcs.compute_nrcs(level1, geo, gmf.nrcs)*1.001
model_rvl_list = [None] * level1.Antenna.size
for aa, ant in enumerate(level1.Antenna.data):
    model_rvl_list[aa] = ss.gmfs.doppler.compute_total_surface_motion(level1.sel(Antenna=ant), geo, gmf=gmf.doppler.name)
level1['RVL'] = xr.concat(model_rvl_list, dim='Antenna')*1.001

In [6]:
# Add NaN for RVL for the mid antennas
level1.RVL[1,:,:] = np.full([9,11], np.nan)
level1.RVL[2,:,:] = np.full([9,11], np.nan)


In [7]:
level1

<xarray.Dataset>
Dimensions:              (Antenna: 4, across: 9, along: 11)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    LookDirection        (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4 5 6 7 8
  * along                (along) int64 0 1 2 3 4 5 6 7 8 9 10
  * Antenna              (Antenna) object 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.07128 ... 0.1668
    RVL                  (Antenna, across, along) float64 0.06364 ... -0.5313

In [8]:
model_rvl_list = [None] * level1.Antenna.size
for aa, ant in enumerate(level1.Antenna.data):
    print(aa,ant)
    model_rvl_list[aa] = ss.gmfs.doppler.compute_total_surface_motion(level1.sel(Antenna=ant), geo, gmf='mouche12')
#     model_rvl_list[aa] = ss.gmfs.doppler.compute_wasv(level1.sel(Antenna=ant), geo, gmf='yurovsky19')

0 Fore
1 MidV
2 MidH
3 Aft


In [9]:
model = level1.drop_vars([var for var in level1.data_vars])
model['RVL'] = xr.concat(model_rvl_list, dim='Antenna')
model['Sigma0'] = ss.gmfs.nrcs.compute_nrcs(level1, geo, gmf=gmf.nrcs)

In [10]:
noise = level1.drop_vars([var for var in level1.data_vars])
# noise = level1.drop_vars(['Sigma0','dsig0','RVL','drvl'])
noise['Sigma0'] = level1.Sigma0*0.05
noise['RVL'] = level1.RVL*0.05

In [11]:
res = (model-level1)/noise

In [12]:
np.concatenate(
    (res.Sigma0.sel(across=1, along=1).data, res.RVL.sel(across=1, along=1).data)
)
    

array([-0.01998002, -0.01998002, -0.01998002, -0.01998002, -0.01998002,
               nan,         nan, -0.01998002])

# Test Fun_residual

In [13]:
sl1 = level1.sel(across=1, along=1)
sn = noise.sel(across=1, along=1)

In [14]:
ss.retrieval.cost_function.fun_residual([5,5,1,1], sl1, sn, gmf)

array([   9.01895349,   12.1681688 ,   12.60884007,   16.59575996,
       -691.66097198,    0.        ,    0.        ,   17.09819452])

### test fun_residual with 2D fields

In [15]:
[geo['U'], geo['V']] = ss.utils.tools.windSpeedDir2UV(geo.WindSpeed, geo.WindDirection)
[geo['C_U'], geo['C_V']] = ss.utils.tools.currentVelDir2UV(geo.CurrentVelocity, geo.CurrentDirection)

In [16]:
geo

<xarray.Dataset>
Dimensions:           (across: 9, along: 11)
Coordinates:
  * across            (across) int64 0 1 2 3 4 5 6 7 8
  * along             (along) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    WindSpeed         (across, along) int64 10 10 10 10 10 10 ... 10 10 10 10 10
    WindDirection     (across, along) int64 150 150 150 150 ... 150 150 150 150
    CurrentVelocity   (across, along) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    CurrentDirection  (across, along) int64 150 150 150 150 ... 150 150 150 150
    U                 (across, along) float64 -5.0 -5.0 -5.0 ... -5.0 -5.0 -5.0
    V                 (across, along) float64 8.66 8.66 8.66 ... 8.66 8.66 8.66
    C_U               (across, along) float64 0.5 0.5 0.5 0.5 ... 0.5 0.5 0.5
    C_V               (across, along) float64 -0.866 -0.866 ... -0.866 -0.866

In [17]:
level1

<xarray.Dataset>
Dimensions:              (Antenna: 4, across: 9, along: 11)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    LookDirection        (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4 5 6 7 8
  * along                (along) int64 0 1 2 3 4 5 6 7 8 9 10
  * Antenna              (Antenna) object 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.07128 ... 0.1668
    RVL                  (Antenna, across, along) float64 0.06364 ... -0.5313

In [23]:
ss.retrieval.cost_function.fun_residual([geo.U, geo.V, geo.C_U, geo.C_V], level1, noise, gmf)

array([[[-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -

In [38]:
ss.retrieval.cost_function.fun_residual([geo.U, geo.V, geo.C_U, geo.C_V], level1, noise, gmf).shape

(8, 9, 11)

# Test least_squares with fun_residual

In [19]:
opt = {
        'method': 'trf', # Trust Region Reflective algorithm, particularly suitable for large sparse problems with bounds. Generally robust method.
        'xtol':  1e-1, # Tolerance for termination by the change of the independent variables
        'x_scale': [7,7,.5,.5], # Characteristic scale of each variable.
        'bounds': ([-30,-30,-5,-5], [30,30,5,5]),
    }
x0_variables = [4,4,0,0]

In [20]:
lmout = least_squares(
    ss.retrieval.cost_function.fun_residual,
    x0_variables,
    args=(sl1, sn, gmf),
    **opt
)

In [21]:
lmout

 active_mask: array([0, 0, 0, 0])
        cost: 359.28576479202127
         fun: array([-5.84578011,  0.84954006,  3.50292383, 13.55825809,  7.28518438,
        0.        ,  0.        , 20.84480552])
        grad: array([-1.11615664e-01,  1.29430583e+01, -1.06386357e+03, -2.18671219e+03])
         jac: array([[-5.87770001e+00, -3.71199300e+00,  5.87770045e+00,
         3.71199316e+00],
       [-4.73286354e+00, -1.55214514e+00,  4.73286370e+00,
         1.55214574e+00],
       [-3.78176383e+00, -1.51456218e+00,  3.78176421e+00,
         1.51456273e+00],
       [-1.26884807e+00, -1.57279132e-01,  1.26884818e+00,
         1.57279253e-01],
       [ 0.00000000e+00,  0.00000000e+00, -2.22214300e+02,
        -2.22214300e+02],
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.66203954e+01,
        -2.66203966e+01]])
     message: '

In [35]:
init = [None] * 4
init[0] = {'x0': [1,2,3,4]}

In [36]:
init

[{'x0': [1, 2, 3, 4]}, None, None, None]

In [26]:
level1.sel(Antenna='Fore').unravel

AttributeError: 'Dataset' object has no attribute 'unravel'

-10.348357834950235

In [59]:
list(d)

['Antenna', 'across', 'along']

In [60]:
d

Frozen(SortedKeysDict({'across': 9, 'along': 11, 'Antenna': 4}))

In [61]:
sl1

<xarray.Dataset>
Dimensions:              (Antenna: 4)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (Antenna) int64 30 30 30 30
    AntennaAzimuthImage  (Antenna) int64 45 90 90 135
    LookDirection        (Antenna) int64 45 90 90 135
    Polarization         (Antenna) <U2 'VV' 'VV' 'HH' 'VV'
    across               int64 1
    along                int64 1
  * Antenna              (Antenna) object 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna) float64 0.07263 0.09347 0.08227 0.17
    RVL                  (Antenna) float64 0.06485 nan nan -0.5413